In [1]:
import os
import shutil

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

from datetime import datetime
import patoolib
 
    
#dkaliyev@1cb.kz

In [2]:
#соединение с pgadmin
import psycopg2
import psycopg2.extras as extras
conn = psycopg2.connect(dbname='postgres', user='postgres', password='24270188', host='localhost', port='5432')

In [3]:
#загрузка драйвера для selenium
from webdriver_manager.microsoft import EdgeChromiumDriverManager
driver = webdriver.Edge(EdgeChromiumDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 101.0.1210
[WDM] - Get LATEST edgedriver version for 101.0.1210 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/101.0.1210.53/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\qyerm\.wdm\drivers\edgedriver\win64\101.0.1210.53]


In [ ]:
#webdriver для chrome 

#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())

In [4]:
#выбор столбца ДАТЫ, чтобы узнать последний квартал
exect = 'SELECT date FROM file1'
cursor = conn.cursor()
cursor.execute(exect)
df = pd.DataFrame(cursor.fetchall())

In [27]:
#последний квартал помогает загрузить конкретный квартал
last_quarter = df.iloc[-1, 0]
last_quarter

'1.01.2022'

In [19]:
#current year & quarters
current_year = datetime.now().year
quarter = ['1.04.', '1.07.', '1.10.', '1.01.']

In [20]:
#путь к папке загрузки
downloads_folder = os.getcwd() + '\\Downloads\\'

In [21]:
#мы получаем наш сайт в драйвере
driver.get('https://stat.gov.kz/official/industry/20/statistic/5')

In [22]:
#выбираем язык РУС на сайте
[n for n in driver.find_elements_by_css_selector("button") if n.text == 'РУС'][0].click()

In [23]:
#мы получаем все файлы на этом сайте
driver.find_element_by_class_name('field-periodity').send_keys('квартал')
driver.find_element_by_class_name('field-title').send_keys('Об объемах оказанных услуг в Республике Казахстан')

In [30]:
#quarter + year 
quarter_and_year = [n + str(current_year) for n in quarter]
quarter_and_year[-1] = quarter[-1] + str(current_year + 1)

In [31]:
quarter_and_year

['1.04.2021', '1.07.2021', '1.10.2021', '1.01.2022']

In [32]:
#проверка года последних файлов, если он равен текущему году, то мы загрузим файлы
if driver.find_elements_by_class_name('left')[1].text[-9:-5] == str(current_year):
    #индекс последнего квартала в листе quarter_and_year
    index = quarter_and_year.index(last_quarter)
    
    #получение файлов за текущий год
    driver.find_element_by_class_name('field-period').send_keys(str(current_year))
    driver.find_element_by_class_name('field-periodity').send_keys('квартал')
    driver.find_element_by_class_name('field-title').send_keys('Об объемах оказанных услуг в Республике Казахстан')
    
    length = len(driver.find_elements_by_class_name('arrow'))
    
    #по последнему кварталу мы загрузим файлы за определенный квартал. Например, если последний квартал = 1.07, 
    #то мы загрузим эти кварталы которые больше последнего 1.04+year < 1.07+year < 1.10+year, 1.01+year 
    #удаляем из листа 1.04+year < 1.07+year 
    for n in range(length):
        if n <= index:
            del quarter_and_year[0]
        else:
            (driver.find_elements_by_class_name('arrow')[n]).click()
            
    #после фор луп проверяем есть ли у нас новый квартaл или нет
    if len(quarter_and_year) == 0:
        print('There is no new files')
    else:
        #выбираем последние скачанные файлы по длину листа quarter_and_year
        files_list = os.listdir(downloads_folder)[-len(quarter_and_year):]
    
        #создание новой папки для хранения загруженных файлов
        new_folder = downloads_folder + 'folder\\'
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        
        #копирование файлов в новую папку
        for n in files_list:
            shutil.copy(downloads_folder + n, new_folder)
        
        #удаляем файлы
        for n in files_list:
            os.remove(downloads_folder + n)
        
        #проверка наличия zip-файлов и распаковка zip-файлов
        zip_files = [n for n in files_list if 'rar' in n or 'zip' in n]
        for n in zip_files:
            patoolib.extract_archive(new_folder + n, outdir=new_folder)
        
        #выбор файлов excel в списке
        files_list = [n for n in os.listdir(new_folder) if 'xls' in n or 'xlsx' in n]
        
        #создание фрейма данных для хранения наших файлов
        table1 = pd.DataFrame(columns=['date','services','volume','small','medium', 'large','state','private','overseas'])
        table2 = pd.DataFrame(columns=['date','services','volume', 'volume_of_the_rural_area'])

        #обработка excel файлов и добавление в наш фрейм данных
        for n in range(len(quarter_and_year)):
            xls = pd.ExcelFile(new_folder + files_list[n])
            try:
                df1 = pd.read_excel(xls, '4.').dropna(axis=0)
                df2 = pd.read_excel(xls, '4.продолжение').dropna(axis=0)
                df3 = pd.read_excel(xls, '5.').dropna(axis=0)
                df4 = pd.read_excel(xls, '5.продолжение').dropna(axis=0)
            except ValueError as v:
                df1 = pd.read_excel(xls, '4,').dropna(axis=0)
                df2 = pd.read_excel(xls, '4,продолжение').dropna(axis=0)
                df3 = pd.read_excel(xls, '5,').dropna(axis=0)
                df4 = pd.read_excel(xls, '5,продолжение').dropna(axis=0)
        
            df1 = df1.rename(columns={df1.columns[0]:df2.columns[0]})
            sheet4 = pd.concat([df1, df2], axis=0, ignore_index=True)
    
            df3 = df3.rename(columns={df3.columns[0]:df4.columns[0]})
            sheet5 = pd.concat([df3, df4], axis=0, ignore_index=True)
        
            date_column = pd.Series([quarter_and_year[n]] * sheet5.shape[0])
            concat_sheet = pd.concat([date_column, sheet4.iloc[:,-1], sheet4.iloc[:,1:-1], sheet5.iloc[:,2:-1]], axis=1)

            concat_sheet.columns = ['date','services','volume','small','medium', 'large','state','private','overseas']
            table1 = pd.concat([table1, concat_sheet], axis=0, ignore_index=True)        
            
            table1 = table1.replace('-', 0)
            table1[['volume','small','medium','large','state','private','overseas']] = table1[['volume','small','medium','large','state','private','overseas']].astype(int)
        
            sheet7 = pd.read_excel(xls, '7.').dropna(axis=0)
            sheet7[sheet7.columns[0]] = sheet7[sheet7.columns[-1]]
            sheet7 = sheet7.iloc[:,0:4].drop(columns=[sheet7.columns[2]]).reset_index(drop=True)

            date_column2 = pd.Series([quarter_and_year[n]] * sheet7.shape[0])
            sheet7 = pd.concat([date_column2, sheet7], axis=1, ignore_index=True)
        
            sheet7.columns = ['date','services','volume', 'volume_of_the_rural_area']
            table2 = pd.concat([table2,  sheet7], axis=0, ignore_index=True)  
            
            table2 = table2.replace('-', 0)
            table2[['volume','volume_of_the_rural_area']] = table2[['volume','volume_of_the_rural_area']].astype(int)
            
        #вставка нашего готового фрейма данных в нашу базу данных pgadmin
        def execute_values(conn, df, table):
  
            tuples = [tuple(x) for x in df.to_numpy()]
            cols = ','.join(list(df.columns))
            query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
            cursor = conn.cursor()
            try:
                extras.execute_values(cursor, query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            print("the dataframe is inserted")
            cursor.close()
            
        execute_values(conn, table1, 'file1')
        execute_values(conn, table2, 'file2')
        
        
#если не равен текущему году, то мы выводим текст
else:
    print('There is no current year')

There is no current year


In [30]:
table1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      25 non-null     object
 1   services  25 non-null     object
 2   volume    25 non-null     int32 
 3   small     25 non-null     int32 
 4   medium    25 non-null     int32 
 5   large     25 non-null     int32 
 6   state     25 non-null     int32 
 7   private   25 non-null     int32 
 8   foreign   25 non-null     int32 
dtypes: int32(7), object(2)
memory usage: 1.2+ KB


In [55]:
def execute_values(conn, df, table):
  
            tuples = [tuple(x) for x in df.to_numpy()]
            cols = ','.join(list(df.columns))
            query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
            cursor = conn.cursor()
            try:
                extras.execute_values(cursor, query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            print("the dataframe is inserted")
            cursor.close()
            
execute_values(conn, table1, 'file1')

the dataframe is inserted
